In [1]:
import requests
import json
from sentence_transformers import SentenceTransformer

from os import listdir
from os.path import isfile, join

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import csv
from lyricsgenius import Genius
import re
from string import digits
import sys

import pandas as pd

In [2]:
def getLyrics(artistName, trackName):

    #CREATE THE URL
    baseUrl = "https://api.musixmatch.com/ws/1.1/"

    apiKey = "&apikey=bd88be6fb629c354c5ca281296ac8c47"

    lyricsMatcher = "matcher.lyrics.get"

    formatUrl = "?format=json&callback=callback"

    artistSearch = "&q_artist="

    trackSearch = "&q_track="

    #MAKE THE API CALL
    apiCall = baseUrl + lyricsMatcher + formatUrl + artistSearch + artistName + trackSearch + trackName + apiKey

    request = requests.get(apiCall)

    data = request.json()

    data = data['message']['body']
    
    if data == []:
        return []

    lyrics = data['lyrics']['lyrics_body']
    
    #at this point lyrics is a string and I turn it into a list.
    
    #PROCESS THE DATA
    lyrics = lyrics.split('\n')
    
    lyrics = list(filter(lambda a: a != '', lyrics))
    
    lyrics = lyrics[:-2]
    
    #output = ""
    
    '''
    for sentence in lyrics:
        output = output + sentence + ". "
        
    output = output[:-1]
    '''
    
    return lyrics

In [3]:
def getLyrics2(artistName, trackName):
    token = "CVXKYLMcyZHbTqZA-j1IMHhDVE7W1SAwpjyLBlhh-upOZlG4aVn_0Joc-xZUB8U-"
    
    genius = Genius(token)
    
    song = genius.search_song(trackName, artistName)
    
    #if song == None:
        #return []
    
    lyrics = song.lyrics
    
    startString = song.title + ' Lyrics'
    
    if lyrics.startswith(startString):
        lyrics = lyrics[len(startString):]
        
    if lyrics.startswith('\xa0'):
        raise Exception("xa0 bs")
    
    lyrics = re.sub("[\(\[].*?[\)\]]", "", lyrics)
    
    remove_digits = str.maketrans('', '', digits)
    lyrics = lyrics.translate(remove_digits)

    lyrics = re.sub(r'\n\s*\n', '\n', lyrics)
    
    lyrics = lyrics.split('\n')
    
    if lyrics[0] == '':
        lyrics = lyrics[1:]
        
    lyrics = lyrics[:-1]
    
    return lyrics

In [4]:
#lyrics = getLyrics2("Fort Minor", "Remember The Name       ")
#print(lyrics)

In [5]:
cid="715ae75406e24c24a3641929260ca343"
secret="e60a686258c646b7a4d3f9636b3235db"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [6]:
def getEmbedding(lyrics):
    model = SentenceTransformer('all-MiniLM-L6-v2')

    embeddings = model.encode(lyrics)

    return embeddings

In [7]:
BIGDATA = {}

In [8]:
print('FUCK YOU')

FUCK YOU


In [ ]:
#you can download the dataset here:
#https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge/dataset_files
#you do need an account however.

mypath = '../../musicdata'

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

numDuds = 0

print('chads here')

with open('songs.csv', 'w', newline='') as fCsv:
    print('opened the csv')
    writer = csv.writer(fCsv)
    
    if len(BIGDATA) == 0:
        header = ['Track Name', 'Artist Name', 'Album Name', 'Artist URI', 'Album URI', 'Genre', 'Tempo', 'Danceability', 'Loudness', 'Lyrics', 'Track Uri']
        writer.writerow(header)
        print('wrote the header')

    i = 0
    
    for fileName in onlyfiles:
        print('\ngoing through', fileName)
        #i += 1
        #if i < 5:
            #continue
            
        if len(BIGDATA) - numDuds >= 1000000:
            print('BIGDATA big enough lets break')
            break
                
        f = open(mypath + '/' + fileName)
        
        print('opening file')

        data = json.load(f)

        playlists = data['playlists']
        
        for playlist in playlists:
            print('iterating through playlists')
            tracks = playlist['tracks']
            
            if len(BIGDATA) - numDuds >= 1000000:
                print('BIGDATA big enough lets break')
                f.close()
                break

            for track in tracks:
                print('iterating through tracks')
                uri = track['track_uri']

                if uri in BIGDATA:
                    print('uri is already in big data')
                    continue

                print('creating entry for uri')
                BIGDATA[uri] = {}

                BIGDATA[uri]['trackName'] = track['track_name']
                BIGDATA[uri]['artistName'] = track['artist_name']
                BIGDATA[uri]['albumName'] = track['album_name']

                BIGDATA[uri]['artistUri'] = track['artist_uri']
                BIGDATA[uri]['albumUri'] = track['album_uri']

                success = False
                print('trying spotify connection')
                for i in range(10):
                    try:
                        BIGDATA[uri]['genres'] = sp.artist(BIGDATA[uri]['artistUri'])["genres"]
                        BIGDATA[uri]['tempo'] = sp.audio_features(uri)[0]['tempo']
                        BIGDATA[uri]['danceability'] = sp.audio_features(uri)[0]['danceability']
                        BIGDATA[uri]['loudness'] = sp.audio_features(uri)[0]['loudness']

                        success = True
                    
                    except:
                        print('continue...')
                        continue
                        
                    if success:
                        break
                        
                if not success:
                    numDuds += 1
                    print('spotify connection failed. going next')
                    continue
                
                print('spotify connection successful.')
        
                try:
                    newTrackName = re.sub("[\(\[].*?[\)\]]", "", BIGDATA[uri]['trackName'])
                    BIGDATA[uri]['lyrics'] = getLyrics2(BIGDATA[uri]['artistName'], newTrackName)
                except:
                    numDuds += 1
                    print('Unable to get lyrics. Go next.')
                    continue
                
                row = list(BIGDATA[uri].values())
                row.append(uri)
                
                #writer.writerow(row)
                
                try:
                    writer.writerow(row)
                    print('Successfully written row.')
                except:
                    numDuds += 1
                    print('Unable to write row. Go next')
                    continue
                

                f.close()

chads here
opened the csv
wrote the header

going through mpd.slice.0-999.json
opening file
iterating through playlists
iterating through tracks
creating entry for uri
trying spotify connection
